In [9]:
import pandas as pd
import seaborn as sns
import json

with open('stats_without_preprocessing.json','r') as f:
    stats = json.load(f)
    
df = pd.DataFrame(stats)

with open('stats_with_preprocessing.json','r') as f:
    stats = json.load(f)
preproc_df = pd.DataFrame(stats)

with open('stats.json') as f:
    bert_stats = json.load(f)
bert_df = pd.DataFrame(bert_stats)

In [10]:
def get_embedding(x, idx):
    try:
        return x.split('/')[idx]
    except:
        return '0'

def parse_df(df, proc_label):
    df['name'] = df['test'].apply(lambda x: get_embedding(x, 0))
    df['embedding'] = df['test'].apply(lambda x: get_embedding(x, 3))
    df['model'] = df['test'].apply(lambda x: get_embedding(x, 4).rstrip('.model'))
    df['size'] = df['model'].apply(lambda x: int(''.join(filter(str.isdigit, x))) )
    df['proc_info'] = proc_label
    return df

df = parse_df(df, 'not_preprocessed')
preproc_df = parse_df(preproc_df, 'preprocessed')
bert_df = parse_df(bert_df, 'preprocessed')

In [11]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,4)

dataplot = preproc_df.append(bert_df)
dataplot = dataplot.append(df)

dataplot = dataplot[( dataplot['embedding'].str.contains('0') | dataplot['name'].str.startswith('NILC') )]
dataplot = dataplot[dataplot['lang'].str.contains('ptbr')]
dataplot = dataplot.sort_values(by='pearson', ascending=False)
dataplot = dataplot.rename(columns={"plot_name": "word_embedding"})

dataplot['pearson'] = dataplot['pearson'].apply(lambda x: round(x, 2) )
dataplot = dataplot.reset_index()
dataplot = dataplot[['name', 'embedding', 'model', 'size', 'proc_info', 'pearson']]
dataplot.to_csv('benchmarks.csv', index=False)